## Simple UI for Terraform 

Reference:
* [Jupyter Interact](http://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html#Basic-interact)


`pip3 install ipywidgets
jupyter nbextension enable --py widgetsnbextension`
 
 
* [Terraform Python](https://github.com/beelit94/python-terraform)

In [ ]:
!pip3 install ConfigParser

In [ ]:
!pip3 install python_terraform

In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets import Layout, Button

### Define Terraform Project Parameter

In [ ]:
path = '.'
tfvars_filename = 'terraform.tfvars'
state_file = 'terraform.tfstate'

### Load Terraform Variable Attributes

In [ ]:
def f(x,y,z):
    global path, tfvars_filename,state_file
    path = x
    tfvars_filename = y
    state_file = z

In [ ]:
path_widget = widgets.Textarea(
    value= path,
    placeholder='Type something',
    description='Path:',
    layout=Layout(width='60%', height='30px'),
    disabled=False
)
tfvars_widget = widgets.Textarea(
    value= tfvars_filename,
    placeholder='Type something',
    description='TFVars File:',
    layout=Layout(width='60%', height='30px'),
    disabled=False
)
state_widget = widgets.Textarea(
    value= state_file,
    placeholder='Type something',
    description='State File:',
    layout=Layout(width='60%', height='30px'),
    disabled=False
)
interact(f,x = path_widget, y= tfvars_widget, z=state_widget);

In [ ]:
import configparser
with open(path+'/'+tfvars_filename, 'r') as f:
    config_string = '[dummy_section]\n' + f.read()
config = configparser.ConfigParser()
config.read_string(config_string)
tfconfig_dict_raw = dict(config._sections['dummy_section'])
print(path+'/'+tfvars_filename)
tfconfig_dict_raw

### Build Widgets

In [ ]:
inputWidgets = []
titles = []
tfconfig_dict = {}
for key, value in tfconfig_dict_raw.items():
    if value == 'true':
        inputWidgets.append(widgets.Checkbox(value=True,description=key,disabled=False))
        tfconfig_dict[key] = True
    elif value == 'false':                   
        inputWidgets.append(widgets.Checkbox(value=False,description=key,disabled=False))
        tfconfig_dict[key] = False
    elif "[" in value :      
        a = value.replace('"', '')
        a = a.replace('[', '')
        a = a.replace(']', '')
        a = a.split(',')
        if "disk" in key :
           inputWidgets.append(widgets.SelectMultiple(options=['25','100'],value=a,description=key,disabled=False))
        else:
           inputWidgets.append(widgets.SelectMultiple(options=a,value=a,description=key,disabled=False))
        tfconfig_dict[key] = a
    else:
        a = value.replace('"', '')
        inputWidgets.append(widgets.Text(description=key, value=a,  layout=Layout(width='70%', height='30px')))
        tfconfig_dict[key] = a
    titles.append(key)
tfconfig_dict

#### Use observe

In [ ]:
def data_changed(change):
    #print('change '+change['owner'].description+' to '+ change['new'])
    tfconfig_dict[change['owner'].description]=change['new']
for t in inputWidgets:
    display(t)
    t.observe(data_changed, 'value')

### Terraform Apply

In [ ]:
print(state_file)
print(tfconfig_dict)

In [ ]:
from python_terraform import *
tf = Terraform(working_dir=path)

apply = Button(description='Terraform Apply',
           layout=Layout(width='50%', height='80px'))

def apply_clicked(b):
    print(b.description+ " clicked. Executing...")
    return_code, stdout, stderr = tf.apply(state=state_file,no_color=IsFlagged, refresh=False, var=tfconfig_dict,capture_output=True)
    print(stdout)
    print(stderr)
    print(return_code) 
           
display(apply)
apply.on_click(apply_clicked)


### Terraform Destroy

In [ ]:
from python_terraform import *
tf = Terraform(working_dir=path_widget.value)

destroy = Button(description='Terraform Destroy',
           layout=Layout(width='50%', height='80px'))

def destroy_clicked(b):
    print(b.description+ " clicked. Executing...")
    return_code, stdout, stderr = tf.destroy(state=state_file,no_color=IsFlagged, refresh=False,capture_output=True)
    print(stdout)
    print(stderr)
    print(return_code) 
    
display(destroy)
destroy.on_click(destroy_clicked)
